# Unsupervised Learning

Como pudimos ver, hay muchos modelos usados ​​en la clasificación de trayectorias, en este cuaderno vamos a usar y probar modelos de aprendizaje no supervisado (en caso de que no haya sido lo suficientemente obvio ya)

Primero que todo vamos a cargar los vectores donde se describen las trayectorias por sus características.

In [16]:
import feature_vec as fv

metadata = fv.get_selected_data()
feat_vectors, clss_mask, clss = fv.get_feat_vectors(metadata)

## K-Means

Aplicando el algoritmo K-Means a los datos

In [21]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=5,
                n_init=10,
                init='k-means++',
                tol=1e-4, 
                random_state=0,
                verbose=True,
                algorithm='elkan')

y_pred = kmeans.fit(feat_vectors)
count_dict = {i: [0]*5 for i in range(5)}
clss_count = [0]*5
for i in range(len(y_pred.labels_)):
    count_dict[y_pred.labels_[i]][clss[i]] += 1
            
count_dict

Initialization complete


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6aad2fac10>
Traceback (most recent call last):
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_ca

Iteration 0, inertia 1.4433413952219285e+34.
Iteration 1, inertia 1.2794656627841043e+34.
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 1.6426655731159217e+34.
Iteration 1, inertia 1.3769235948913673e+34.
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 1.3384368582462997e+34.
Iteration 1, inertia 1.2794656627841046e+34.
Converged at iteration 1: strict convergence.


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6ad38c7f70>
Traceback (most recent call last):
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_ca

Initialization complete
Iteration 0, inertia 1.5446985858279894e+34.
Iteration 1, inertia 1.3769235948913675e+34.
Converged at iteration 1: strict convergence.
Initialization complete


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6ad33be430>
Traceback (most recent call last):
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_ca

Iteration 0, inertia 1.5446985858279633e+34.
Iteration 1, inertia 1.376923594891367e+34.
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 1.338436858246307e+34.
Iteration 1, inertia 1.2794656627841046e+34.
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 1.5117608176901293e+34.
Iteration 1, inertia 1.3769235948913673e+34.
Converged at iteration 1: strict convergence.


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6ab38c1820>
Traceback (most recent call last):
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/manoly/.local/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_ca

Initialization complete
Iteration 0, inertia 1.5446985858279912e+34.
Iteration 1, inertia 1.3769235948913673e+34.
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 1.5117608176901572e+34.
Iteration 1, inertia 1.3769235948913673e+34.
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 1.5446985858279924e+34.
Iteration 1, inertia 1.3769235948913673e+34.
Converged at iteration 1: strict convergence.


{0: [1174, 408, 294, 986, 1072],
 1: [0, 1, 0, 0, 0],
 2: [1, 0, 0, 0, 0],
 3: [2, 0, 1, 1, 2],
 4: [1, 0, 0, 0, 0]}

Ahora, determinemos la homogeneidad, la integridad de los clusters y el Coeficiente de Silhouette.

In [15]:
from sklearn import metrics

# Calcular la homogeneidad y la integridad de los clusters.
homogeneity = metrics.homogeneity_score(clss, y_pred)
completeness = metrics.completeness_score(clss, y_pred) 

# Calcular el coeficiente de coeficiente de Silhouette para cada muestra.
s = metrics.silhouette_samples(feat_vectors, y_pred)

# Calcule el coeficiente de Silhouette medio de todos los puntos de datos.
s_mean = metrics.silhouette_score(feat_vectors, y_pred)

print(homogeneity,
    completeness,
    s,
    s_mean)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2521072/1429499525.py", line 4, in <module>
    homogeneity = metrics.homogeneity_score(clss, y_pred)
  File "/home/manoly/.local/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py", line 554, in homogeneity_score
    return homogeneity_completeness_v_measure(labels_true, labels_pred)[0]
  File "/home/manoly/.local/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py", line 461, in homogeneity_completeness_v_measure
    labels_true, labels_pred = check_clusterings(labels_true, labels_pred)
  File "/home/manoly/.local/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py", line 57, in check_clusterings
    type_pred = type_of_target(labels_pred)
  File "/home/manoly/.local/lib/python3.8/site-packages/sklearn/utils/multiclass.py", l

TypeError: object of type 'NoneType' has no len()

## DBSCAN

Aplicamos ahora el algoritmo DBSCAN como otro modelo alternativo.

In [15]:
from sklearn.cluster import DBSCAN

Usamos `SatandarScaler` para estandarizar características a media cero y varianza unitaria. Luego aplicamos el algoritmo a los datos.

In [26]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(feat_vectors)

dbscan = DBSCAN(eps=0.3, min_samples=30).fit_predict(X)

Y ahora calculamos e imprimimos las evaluaciones de las métricas.

In [27]:
print('Number of clusters: {}'.format(len(set(dbscan[np.where(dbscan != -1)]))))
print('Homogeneity: {}'.format(metrics.homogeneity_score(clss, dbscan)))
print('Completeness: {}'.format(metrics.completeness_score(clss, dbscan)))
# print('Mean Silhouette score: {}'.format(metrics.silhouette_score(X, dbscan)))


Number of clusters: 0
Homogeneity: 3.350995561647498e-16
Completeness: 1.0
